In [46]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import os
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# Read CSV lat/long
lat_long_df = pd.read_csv('fatal_alligator_attacks_US_with_latlng.csv')
lat_long_df.head()

,Date,First Name,Last Name,Age,Sex,City,State,Lat,Lng,Details
0,8/15/2022,Nancy,Becker,88,female,Hilton Head Island,South Carolina,32.2163,-80.7526,The victim fell into a pond in a gated communi...
1,7/15/2022,Rose Marie,Wiegand,80s,female,Englewood,Florida,26.9620,-82.3526,The victim fell into a pond along a golf cours...
2,6/24/2022,Micheal,Burstein,75,male,Myrtle Beach,South Carolina,33.6891,-78.8867,The victim was dragged into a pond in Myrtle B...
3,5/31/2022,Sean,McGuinness,47,male,Largo,Florida,27.9095,-82.7873,The victim was attacked while trying to retrie...
4,8/30/2021,Timothy,Satterlee,71,male,Slidell,Louisiana,30.2752,-89.7812,The victim was attacked in floodwaters of Hurr...


In [47]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = lat_long_df['Lat']
lngs = lat_long_df['Lng']
lat_lngs = list(zip(lats, lngs))
lats_lngs_df = pd.DataFrame(lat_lngs, columns = ['Lat', 'Lng'])

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(cities)

['hilton head island', 'englewood', 'myrtle beach', 'largo', 'slidell', 'charleston', 'davie', 'doctor phillips', 'lakeland', 'micco', 'de land', 'orange', 'tamiami', 'wilmington island', 'eustis', 'oldsmar', 'tamarac', 'port charlotte', 'cypress lake', 'kissimmee', 'lynn haven', 'iona', 'winter haven', 'venice', 'deltona', 'zephyrhills', 'hobe sound', 'jasmine estates', 'tallahassee', 'west palm beach', 'fort pierce', 'lake wales', 'laurel', 'gola gokarannath', 'baytown', 'natchitoches', 'fredericksburg']


In [48]:
url='http://api.openweathermap.org/data/2.5/weather?'
units='imperial'
city_url=f'{url}appid={weather_api_key}&units={"imperial"}&q='

name=[]
cloudiness=[]
country=[]
date=[]
humidity=[]
latitude=[]
longitude=[]
maxtemp=[]
wind=[]
count=1

print('Beginning Data Retrieval')     
print('-----------------------------')
for city in cities:
    response=requests.get(city_url + city).json()
        
    print('Processing Record ' + str(count) + ' of ' + str(len(cities)) + ' | ' + city)
        
    try:
        name.append(response['name'])
        cloudiness.append(response['clouds']['all'])
        country.append(response['sys']['country'])
        date.append(response['dt'])
        humidity.append(response['main']['humidity'])             
        latitude.append(response['coord']['lat'])
        longitude.append(response['coord']['lon'])             
        maxtemp.append(response['main']['temp_max'])
        wind.append(response['wind']['speed'])      
    except KeyError:
            
            print(city + ' city not found. Skipping...')    
            count=count+1

Beginning Data Retrieval
-----------------------------
Processing Record 1 of 37 | hilton head island
Processing Record 1 of 37 | englewood
Processing Record 1 of 37 | myrtle beach
Processing Record 1 of 37 | largo
Processing Record 1 of 37 | slidell
Processing Record 1 of 37 | charleston
Processing Record 1 of 37 | davie
Processing Record 1 of 37 | doctor phillips
Processing Record 1 of 37 | lakeland
Processing Record 1 of 37 | micco
Processing Record 1 of 37 | de land
Processing Record 1 of 37 | orange
Processing Record 1 of 37 | tamiami
Processing Record 1 of 37 | wilmington island
Processing Record 1 of 37 | eustis
Processing Record 1 of 37 | oldsmar
Processing Record 1 of 37 | tamarac
Processing Record 1 of 37 | port charlotte
Processing Record 1 of 37 | cypress lake
Processing Record 1 of 37 | kissimmee
Processing Record 1 of 37 | lynn haven
Processing Record 1 of 37 | iona
Processing Record 1 of 37 | winter haven
Processing Record 1 of 37 | venice
Processing Record 1 of 37 | del

In [49]:
columns_dict = {
    'City': name,
    'Lat': latitude,
    'Lng': longitude,
    'Max Temp (F)': maxtemp,
    'Humidity (%)': humidity,
    'Cloudiness (%)': cloudiness,
    'Wind Speed (mph)': wind,
    'Country': country,
    'Date': date}
weather_df=pd.DataFrame(columns_dict)
weather_df.count()
weather_df.to_csv(output_data_file, index=False)
weather_df.head()

,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,Hilton Head,32.2163,-80.7526,51.98,53,0,8.05,US,1671488147
1,Englewood,39.6478,-104.9878,49.41,12,75,17.27,US,1671488219
2,Myrtle Beach,33.6891,-78.8867,49.03,62,0,0.00,US,1671488126
3,Largo,27.9095,-82.7873,72.14,53,0,7.00,US,1671487968
4,Slidell,30.2752,-89.7812,50.02,87,100,10.36,US,1671488199
